# Calculate low and high stock solutions concentrations given standard media recipe

This notebook calculates stock concentrations for a media optimization project. Given a standard media recipe, ranges for intervals to be explored, the notebook generates sets of low and high concentrations such that media preparation can be done without dilutions (which reduces the number of operations, hence time, and the number of pipette tips needed). 


Tested using **ART 3.9.4** kernel on jprime.lbl.gov

## Inputs and outputs

#### Required files to run this notebook:
   - `../data/flaviolin/standard_recipe_concentrations_extended.csv`
   
   A file with the standard media recipe. This file also contains a column with **solubility limits** for each component. 
   
An example of the file content:

| Component | Concentration   | Solubility
|------|------|------|
|   MOPS[mM]  | 40 | 2389.37 |
| H3BO3[mM] | 0.004 | 700 |
| K2SO4[mM] | 0.29 | 636.98 |

   - `../data/flaviolin/Putida_media_bounds_extended.csv`
   
   A file containing upper and lower bounds for media components to be explored.


#### Files generated by running this notebook:

   - `stock_concentrations.csv`
    
stored in the user defined directory.

## Setup

Importing needed libraries:

In [1]:
import sys
sys.path.append('../')

import string
import pandas as pd
import numpy as np
import scipy

from pyDOE import lhs

import core

### User parameters

In [2]:
user_params = {
    'standard_media_file': '../data/flaviolin/standard_recipe_concentrations_extended.csv',  
    'output_file_path': '../data/flaviolin/', # Folder for output files
    'stock_conc_filename': 'stock_concentrations_extended.csv', # Name of the file containing stock concentrations
    'factor_range': 10,             # How many times higher/lower values from the 
    # standard media you want to explore? If you want to explore different 
    # relative ranges across components, you can specify it below (see cell 6)
    'bounds_file': '../data/flaviolin/Putida_media_bounds_extended.csv', # name of the file with bounds needed for ART
    'well_volume': 1500,            # Total volume of the media content+culture in the well
    'min_volume_transfer': 5,       # Minimal transfer volume of the liquid handler
    'culture_factor': 100,          # Dilution factor for culture, e.g. 100x, 1000x
    } 

In [3]:
culture_volume = user_params['well_volume'] / user_params['culture_factor']


Read the standard media recipe concentrations

In [4]:
df_stand = pd.read_csv(user_params['standard_media_file'])
df_stand = df_stand.set_index("Component")
df_stand

,Concentration,Solubility
Component,,
MOPS[mM],40.00000,2389.37
Tricine[mM],4.00000,500.08
H3BO3[mM],0.00400,564.45
Glucose[mM],20.00000,5045.63
K2SO4[mM],0.29000,636.98
K2HPO4[mM],1.32000,8564.84
FeSO4[mM],0.01000,920.80
NH4Cl[mM],9.52000,5290.71
MgCl2[mM],0.52000,555.61


In [5]:
if 'bounds_file' in user_params:
    df_bounds = pd.read_csv(user_params['bounds_file'])
    df_bounds = df_bounds.set_index("Variable")
    display(df_bounds)
else:
    print("Please provide the correct path to bounds file.")

,Min,Max
Variable,,
MOPS[mM],40.000000,40.0000
Tricine[mM],4.000000,4.0000
H3BO3[mM],0.000400,0.0800
Glucose[mM],20.000000,200.0000
K2SO4[mM],0.010000,1.0000
K2HPO4[mM],0.264000,13.2000
FeSO4[mM],0.001000,0.1000
NH4Cl[mM],6.400000,47.6000
MgCl2[mM],0.026000,2.6000


## Find a set of low level stock concentrations that can achieve the lowest levels of target concentrations

$$c_s=\frac{c_{t_{\min}} \cdot V_\text{well}}{V_{\min}}$$

In [6]:
min_tip_volume = user_params['min_volume_transfer']
df_low = pd.DataFrame(
    index=df_stand.index,
    columns=["Stock Concentration", "Target Concentration"])
df_low["Target Concentration"] = df_bounds['Min']
df_low["Stock Concentration"] = df_low["Target Concentration"]*user_params['well_volume']/min_tip_volume
df_low

,Stock Concentration,Target Concentration
Component,,
MOPS[mM],12000.0000,40.000000
Tricine[mM],1200.0000,4.000000
H3BO3[mM],0.1200,0.000400
Glucose[mM],6000.0000,20.000000
K2SO4[mM],3.0000,0.010000
K2HPO4[mM],79.2000,0.264000
FeSO4[mM],0.3000,0.001000
NH4Cl[mM],1920.0000,6.400000
MgCl2[mM],7.8000,0.026000


### Check solubility 

Increase the volume transfer, in increments of 5uL, for the components for which concenstrations are not soluble (there is no need to make minimal volume transfers)

$$c^i_{s}=\frac{c^i_{t_{\min}} \cdot V_\text{well}}{V_{\min}+5}$$

In [7]:
if 'Solubility' in df_stand.columns:
    
    nonsol_comp_low = core.check_solubility(df_low, solubility=df_stand['Solubility'])
    volume_transfer = min_tip_volume

    i = 0
    while len(nonsol_comp_low) > 0:    
        print(f'  Iteration {i}\n')
        volume_transfer += min_tip_volume

        for comp in nonsol_comp_low:
            df_low.at[comp,"Stock Concentration"] = df_low.at[
                comp,"Target Concentration"
            ]*user_params['well_volume']/volume_transfer

        nonsol_comp_low = core.check_solubility(df_low, solubility=df_stand['Solubility'])
        i += 1
    
else:
    print('Solubility values are not provided and it is assumed the limits are not reached.')
    

Components for which those concentrations are not soluble:
	MOPS[mM]
	Tricine[mM]
	Glucose[mM]
  Iteration 0

Components for which those concentrations are not soluble:
	MOPS[mM]
	Tricine[mM]
  Iteration 1

Components for which those concentrations are not soluble:
	MOPS[mM]
  Iteration 2

Components for which those concentrations are not soluble:
	MOPS[mM]
  Iteration 3

Components for which those concentrations are not soluble:
	MOPS[mM]
  Iteration 4



Check if all volumes are larger than the minimal transfer volume (5 uL)

In [8]:
EPS = 0.000001
volumes, df = core.find_volumes(
    user_params['well_volume'], 
    components=df_low.index,
    stock_conc_val=df_low['Stock Concentration'].values, 
    target_conc_val=df_low['Target Concentration'].values,
    culture_ratio=user_params['culture_factor']
)
assert (df['Volumes[uL]'].values >= min_tip_volume - EPS).all(), f"Not all volumes are >={min_tip_volume}uL!"

In [9]:
df

,Stock Concentration,Target Concentration,Volumes[uL]
Component,,,
MOPS[mM],2000.0000,40.000000,30.0
Tricine[mM],400.0000,4.000000,15.0
H3BO3[mM],0.1200,0.000400,5.0
Glucose[mM],3000.0000,20.000000,10.0
K2SO4[mM],3.0000,0.010000,5.0
K2HPO4[mM],79.2000,0.264000,5.0
FeSO4[mM],0.3000,0.001000,5.0
NH4Cl[mM],1920.0000,6.400000,5.0
MgCl2[mM],7.8000,0.026000,5.0


Round to 6 digits after decimal point

In [10]:
num_digits = 6
conc = np.array([round(num, num_digits) for num in list(df_low['Stock Concentration'].values)])
df_low['Stock Concentration'] = conc


## Find a set of high level stock concentrations that can achieve the highest levels of target concentrations

Find stock concentrations for the upper limit in the range to explore.

In [11]:
df_high = df_low.copy()
df_high["Target Concentration"] = df_bounds['Max']
df_high["Solubility"] = df_stand['Solubility']

Check if there are feasible volumes for the low level concentrations found above:

In [12]:
try:
    volumes, df = core.find_volumes(
        user_params['well_volume'],
        components=df_high.index,
        stock_conc_val=df_high['Stock Concentration'].values, 
        target_conc_val=df_high['Target Concentration'].values,
        culture_ratio=user_params['culture_factor']
    )
    feasible_volumes = True
    assert (df['Volumes[uL]'].values >= min_tip_volume - EPS).all(), f"Not all volumes are >={min_tip_volume}uL!"
except AssertionError:
    feasible_volumes = False
    print(core.NoFeasibleVolumesWarn())
    

No feasible volumes are found!


### Find feasible volumes

Increase the current stock concentrations, by 5-fold increments, of components which are the furthest away from the solubility limit  

In [13]:
if not feasible_volumes:
    print("No feasible volumes")
    
    MULTIPL_FACTOR = 5

    success = False
    df = df_high.copy()

    i = 0
    while success is False:
        i += 1
        comp = None

        # Find ratios of solubility over current stock concentrations
        df['Ratio'] = df['Solubility'].values / df['Stock Concentration'].values
        
        # Find which component is the furthest away from the solubility limit
        while comp is None:
            if any(df['Ratio'] > MULTIPL_FACTOR):
                comp = df['Ratio'].idxmax()
            else:
                MULTIPL_FACTOR /= 2
        
        # Increase the current stock concentration by a factor
        df.at[comp, 'Stock Concentration'] *= MULTIPL_FACTOR

        # Find if there are feasible volumes for such stock and target concentrations
        try:
            volumes, df_high = core.find_volumes(
                user_params['well_volume'], 
                components=df.index,
                stock_conc_val=df['Stock Concentration'].values, 
                target_conc_val=df['Target Concentration'].values,
                culture_ratio=user_params['culture_factor']
            )
            success = True
            if success:
                print(f'Iteration {i}:')
                print('Success!')
        except:
            pass
else:
    df_high = df.copy()
    
df_high["Solubility"] = df_stand['Solubility']


No feasible volumes
Iteration 64:
Success!


See what are the calculated volumes

In [14]:
df_high

,Stock Concentration,Target Concentration,Volumes[uL],Solubility
Component,,,,
MOPS[mM],2000.0000,40.0000,30.00000,2389.37
Tricine[mM],400.0000,4.0000,15.00000,500.08
H3BO3[mM],375.0000,0.0800,0.32000,564.45
Glucose[mM],3000.0000,200.0000,100.00000,5045.63
K2SO4[mM],375.0000,1.0000,4.00000,636.98
K2HPO4[mM],4950.0000,13.2000,4.00000,8564.84
FeSO4[mM],468.7500,0.1000,0.32000,920.80
NH4Cl[mM],1920.0000,47.6000,37.18750,5290.71
MgCl2[mM],195.0000,2.6000,20.00000,555.61


### Correct for minimal transfer volumes

If there are volumes that are smaller than the minimum transfer volume, change stock concentrations for those components (decrease the concentrations so that the volume increases).

In [15]:
# Find components with volume transfers smaller than the minimal
comp_small_vol = df_high[
    df_high['Volumes[uL]'] < min_tip_volume - EPS
].index
print(f"{len(comp_small_vol)} component(s) found with volume transfers smaller than the minimal")

# Define new volume transfer to be higher than the minimal, so there is some flexibility
NEW_VOLUME_TRANSFER = 5.0*min_tip_volume

for comp in comp_small_vol:
    factor_diff =  NEW_VOLUME_TRANSFER / (df_high.at[comp, 'Volumes[uL]'])
    print(f'Decreasing the concentration of {comp} by {factor_diff} times')
    df_high.at[comp, 'Stock Concentration'] /= factor_diff
    

10 component(s) found with volume transfers smaller than the minimal
Decreasing the concentration of H3BO3[mM] by 78.125 times
Decreasing the concentration of K2SO4[mM] by 6.25 times
Decreasing the concentration of K2HPO4[mM] by 6.25 times
Decreasing the concentration of FeSO4[mM] by 78.125 times
Decreasing the concentration of (NH4)6Mo7O24[mM] by 9765.625 times
Decreasing the concentration of CoCl2[mM] by 1953.125 times
Decreasing the concentration of CuSO4[mM] by 3255.208333333333 times
Decreasing the concentration of MnSO4[mM] by 1953.125 times
Decreasing the concentration of ZnSO4[mM] by 9765.625 times
Decreasing the concentration of CaCl2[mM] by 1953.125 times


Recalculate volumes for corrected stock concentrations:

In [16]:
volumes, df_high_new = core.find_volumes(
    user_params['well_volume'], 
    components=df_high.index,
    stock_conc_val=df_high['Stock Concentration'].values, 
    target_conc_val=df_high['Target Concentration'].values,
    culture_ratio=user_params['culture_factor']
)
df_high_new

,Stock Concentration,Target Concentration,Volumes[uL]
Component,,,
MOPS[mM],2000.000,40.0000,30.0000
Tricine[mM],400.000,4.0000,15.0000
H3BO3[mM],4.800,0.0800,25.0000
Glucose[mM],3000.000,200.0000,100.0000
K2SO4[mM],60.000,1.0000,25.0000
K2HPO4[mM],792.000,13.2000,25.0000
FeSO4[mM],6.000,0.1000,25.0000
NH4Cl[mM],1920.000,47.6000,37.1875
MgCl2[mM],195.000,2.6000,20.0000


Round to 5 digits after decimal point

In [17]:
df_high = df_high_new.copy()
num_digits = 5
conc = np.array([round(num, num_digits) for num in list(df_high['Stock Concentration'].values)])
df_high['Stock Concentration'] = conc


Create the final dataframe with low and high concentrations and dilution factor for their preparation

In [18]:
df_stock = df_low.copy()
df_stock.rename(columns={'Stock Concentration': 'Low Concentration'}, inplace=True)
df_stock = df_stock.drop(['Target Concentration'], axis='columns')
df_stock['High Concentration'] = df_high['Stock Concentration']
df_stock['Dilution Factor'] = df_stock['High Concentration']/df_stock['Low Concentration']
df_stock

,Low Concentration,High Concentration,Dilution Factor
Component,,,
MOPS[mM],2000.0000,2000.000,1.0
Tricine[mM],400.0000,400.000,1.0
H3BO3[mM],0.1200,4.800,40.0
Glucose[mM],3000.0000,3000.000,1.0
K2SO4[mM],3.0000,60.000,20.0
K2HPO4[mM],79.2000,792.000,10.0
FeSO4[mM],0.3000,6.000,20.0
NH4Cl[mM],1920.0000,1920.000,1.0
MgCl2[mM],7.8000,195.000,25.0


### Test found stock concentrations for different, randomly chosen, target concentrations

Create random target concentrations, sampled using Latin Hypercube, given lower/upper bounds:

In [19]:
n_samples = 1000

latin_hc = lhs(
    len(df_stock), samples=n_samples, criterion="maximin"
)

lb = df_bounds['Min'].ravel()
ub = df_bounds['Max'].ravel()

target_conc_val = lb + latin_hc * (ub - lb)

df_target_conc = pd.DataFrame(
    data=target_conc_val, 
    columns=df_stock.index
)

Check what are the volumes for random choices of target concentrations within the given ranges:

In [20]:
%%time

df_volumes = core.find_volumes_bulk(
    df_stock, 
    df_target_conc=df_target_conc, 
    well_volume=user_params['well_volume'],
    min_tip_volume=min_tip_volume,
    culture_ratio=user_params['culture_factor'],
    verbose=0
)

Sucess rate: 93.1%
Sucess rate (water): 92.3%
CPU times: user 15.6 s, sys: 31.3 s, total: 46.9 s
Wall time: 4.7 s


### Save the file with stock concentrations

In [21]:
stock_conc_file = f'{user_params["output_file_path"]}/{user_params["stock_conc_filename"]}.csv'

In [22]:
df_stock.to_csv(stock_conc_file)